<a href="https://colab.research.google.com/github/auramolina/Analitica-en-recursos-humanos/blob/main/Modelado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='0C2054'><b>Librerías

In [58]:
#### Cargar paquetes
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt

from sklearn import linear_model, tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate, RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, get_scorer_names, accuracy_score,confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler ## escalar variables

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

#metrics
from sklearn.metrics import accuracy_score, classification_report

import joblib  ### para guardar modelos
import openpyxl
import sys
import os

In [59]:
## Conectar el drive
from google.colab import drive
drive.mount('/content/drive')

path='/content/drive/MyDrive/AA/Analitica-en-recursos-humanos'
os.chdir(path) ### setting a Working directory on drive path
sys.path.append(path+'/')  ###setting path to read user defined functions

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
import funciones_jc as fn ### importar modulo de  funciones
from funciones_jc import * ### importar toddas llas funciones del archivo

## con esto ya ses pueden utilizar todas las funciones propias

In [ ]:
d2015 = pd.read_csv('https://raw.githubusercontent.com/auramolina/Analitica-en-recursos-humanos/main/Datos/dummies2015')
d2015

#<font color='0C2054'><b> Separación variables

In [ ]:
### Variable respuesta
y = d2015['Attrition']

## Variables explicativas
X1 = d2015.loc[:,~d2015.columns.isin(['Attrition'])]

## <font color='0C2054'>Estandarización

In [ ]:
## Estandarización y escalado de los datos
scaler = StandardScaler()
scaler.fit(X1)
X2 = scaler.transform(X1)
X = pd.DataFrame(X2,columns=X1.columns)

# <font color='0C2054'><b>Modelos Iniciales</font></b>

* Regresión logística: se selecciona LR por su fácil implementación, no requiere ajuste de hiperparametros y, es bueno en problemas de clasificación binaria.
* Bosques aleatorios: se selecciona RFC por su rendimiento, que generalmente es bueno, dando resultados satisfactorios, también porque es bueno en modelos que tienen muchas variables categóricas.

In [ ]:
m_lr = LogisticRegression()
m_rf = RandomForestClassifier()
m_tr = DecisionTreeClassifier()
#m_gbt = GradientBoostingClassifier()

In [ ]:
modelos = list([m_lr, m_rf, m_tr])

## <font color='0C2054'>Selección de variables

In [ ]:
var_names = fn.sel_variables(modelos, X, y, threshold = "2*mean")
var_names.shape

In [ ]:
X2 = X[var_names] ### matriz con variables seleccionadas
X2.info()
X.info()

# <font color='0C2054'><b>Modelado

## <font color='0C2054'>Calculo F1-score

In [63]:
def medir_modelos(modelos,scoring,X,y,cv):

    metric_modelos=pd.DataFrame()
    for modelo in modelos:
        scores=cross_val_score(modelo,X,y, scoring=scoring, cv=cv )
        pdscores=pd.DataFrame(scores)
        metric_modelos=pd.concat([metric_modelos,pdscores],axis=1)

    metric_modelos.columns=["reg_lineal","decision_tree","random_forest"]
    return metric_modelos

In [64]:
# Calcular F1-score con 30 iteraciones para todos los modelos con todas las variables
f1score_df = medir_modelos(modelos, "f1", X, y, 30)
# Calcular F1-score con 30 iteraciones para todos los modelos con variables seleccionadas
f1_score_varsel = medir_modelos(modelos, "f1", X2, y, 30)
# visualizar los resultados de f1- score para los modelos con todas las variabels y con las variables seleccionadas
f1=pd.concat([f1score_df,f1_score_varsel],axis=1)
f1.columns=['rl', 'dt', 'rf', 'rl_Sel', 'dt_sel', 'rf_sel']
f1

,rl,dt,rf,rl_Sel,dt_sel,rf_sel
0,0.370370,0.975610,0.975610,0.000000,0.975610,0.975610
1,0.320000,1.000000,1.000000,0.090909,1.000000,1.000000
2,0.230769,1.000000,0.975610,0.000000,1.000000,0.975610
3,0.230769,1.000000,1.000000,0.173913,1.000000,1.000000
4,0.086957,1.000000,1.000000,0.000000,1.000000,1.000000
5,0.074074,1.000000,1.000000,0.000000,1.000000,1.000000
6,0.320000,1.000000,1.000000,0.000000,0.975610,0.975610
7,0.240000,1.000000,1.000000,0.000000,1.000000,1.000000
8,0.214286,0.764706,0.551724,0.086957,0.687500,0.551724
9,0.083333,0.250000,0.000000,0.000000,0.500000,0.000000


In [65]:
# promedio de la metrica F1-score para todos los modelos con todas las variables y con las variables seleccionadas
f1.mean()

,0
rl,0.212993
dt,0.851468
rf,0.790158
rl_Sel,0.034673
dt_sel,0.839631
rf_sel,0.788356


## <font color='0C2054'>Grilla de hiperparametros para el modelo Ramdon Forest

In [66]:
param_grid_rf = {
    'n_estimators': [5, 50, 100, 200, 500],  # Número de árboles en el bosque
    'max_features': [5, 20],  # Número de características a considerar en cada división
    'max_depth': [10, 20, 30],  # Profundidad máxima del árbol
    'min_samples_split': [5, 20, 100],  # Número mínimo de muestras requeridas para dividir un nodo interno
    'min_samples_leaf': [1, 2, 4],  # Número mínimo de muestras requeridas para estar en un nodo hoja
    'bootstrap': [True, False]  # Método de muestreo para construir árboles
}

## <font color='0C2054'>Grilla de hiperparámetros para Gradient Boosting

In [67]:
param_grid_gb = {
    'n_estimators': [100, 200, 500],  # Número de árboles en el ensemble
    'learning_rate': [0.01, 0.1, 0.2, 0.3],  # Tasa de aprendizaje
    'max_depth': [3, 5, 8],  # Profundidad máxima de los árboles
    'min_samples_split': [2, 5, 10],  # Mínimo número de muestras necesarias para dividir un nodo
    'min_samples_leaf': [1, 2, 4],  # Mínimo número de muestras necesarias en un nodo hoja
    'subsample': [0.8, 0.9, 1.0],  # Fracción de muestras utilizadas para entrenar cada árbol
    'max_features': ['auto', 'sqrt', 'log2']  # Número de características a considerar al hacer cada división
}

## <font color='0C2054'>Técnica de búsqueda de hiperparámetros para el modelo Ramdon Forest

In [68]:
tun_rf = RandomizedSearchCV(m_rf,param_distributions=param_grid_rf,n_iter=6,scoring="f1")
tun_rf.fit(X2,y)

RandomizedSearchCV(estimator=RandomForestClassifier(), n_iter=6,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30],
                                        'max_features': [5, 20],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [5, 20, 100],
                                        'n_estimators': [5, 50, 100, 200, 500]},
                   scoring='f1')

In [70]:
# Imprimir los parametros de la grilla con el mejor puntaje
pd.set_option('display.max_colwidth', 100)
resultados=tun_rf.cv_results_
tun_rf.best_params_
pd_resultados=pd.DataFrame(resultados)
pd_resultados[["params","mean_test_score"]].sort_values(by="mean_test_score", ascending=False)

,params,mean_test_score
2,"{'n_estimators': 500, 'min_samples_split': 20, 'min_samples_leaf': 1, 'max_features': 20, 'max_d...",0.661403
5,"{'n_estimators': 200, 'min_samples_split': 20, 'min_samples_leaf': 1, 'max_features': 5, 'max_de...",0.612786
4,"{'n_estimators': 5, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 5, 'max_depth...",0.600668
0,"{'n_estimators': 200, 'min_samples_split': 20, 'min_samples_leaf': 4, 'max_features': 5, 'max_de...",0.587834
1,"{'n_estimators': 500, 'min_samples_split': 100, 'min_samples_leaf': 1, 'max_features': 5, 'max_d...",0.299139
3,"{'n_estimators': 500, 'min_samples_split': 100, 'min_samples_leaf': 2, 'max_features': 5, 'max_d...",0.198769


## <font color='0C2054'>Técnica de búsqueda de hiperparámetros para el GradientBoostingClassifier

In [72]:
# Crear la instancia de GridSearchCV para búsqueda de hiperparámetros
#tun_gb = GridSearchCV(estimator=m_gbt, param_grid=param_grid_gb, scoring='f1', cv=5)
# Realizar la búsqueda de hiperparámetros en los datos
#tun_gb.fit(X2, y)

In [ ]:
# Imprimir los parametros de la grilla con el mejor puntaje
#pd.set_option('display.max_colwidth', 100)
#resultados=tun_gb.cv_results_
#tun_gb.best_params_
#pd_resultados=pd.DataFrame(resultados)
#pd_resultados[["params","mean_test_score"]].sort_values(by="mean_test_score", ascending=False)

In [75]:
# Mejor grilla del modelo GradientBoostingClassifier
#dt_final = tun_gb.best_estimator_

# Guardar el modelo con hyperparameter tunning
rf_final = tun_rf.best_estimator_

# <font color='0C2054'><b>Analisis del los modelos

In [76]:
#####Evaluar métrica de entrenamiento y evaluación para mirar sobre ajuste ####
#eval_gb=cross_validate(gb_final,X2,y,cv=30,scoring="f1",return_train_score=True)
eval_rf=cross_validate(rf_final,X2,y,cv=30,scoring="f1",return_train_score=True)

KeyboardInterrupt: 

In [ ]:
#### convertir resultado de evaluacion entrenamiento y evaluacion en data frame para Ramdon Forest
train_rf=pd.DataFrame(eval_rf['train_score'])
test_rf=pd.DataFrame(eval_rf['test_score'])
train_test_rf=pd.concat([train_rf, test_rf],axis=1)
train_test_rf.columns=['train_score','test_score']
train_test_rf

In [ ]:
#### convertir resultado de evaluacion entrenamiento y evaluacion en data frame para Ramdon Forest
train_gb=pd.DataFrame(eval_gb['train_score'])
test_gb=pd.DataFrame(eval_gb['test_score'])
train_test_gb=pd.concat([train_gb, test_gb],axis=1)
train_test_gb.columns=['train_score','test_score']
train_test_gb